In [2]:
!pip install plotly
!pip install boto3==1.19.12
!pip install s3fs
!pip install lightgbm
!pip install shap
!pip install catboost


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
  Using cached boto3-1.19.12-py3-none-any.whl.metadata (6.4 kB)
  Using cached botocore-1.22.12-py3-none-any.whl.metadata (5.6 kB)
  Using cached s3transfer-0.5.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached urllib3-1.26.18-py2.py3-none-any.whl.metadata (48 kB)
Using cached boto3-1.19.12-py3-none-any.whl (131 kB)
Using cached botocore-1.22.12-py3-none-any.whl (8.1 MB)
Using cached s3transfer-0.5.2-py3-none-any.whl (79 kB)
Using cached urllib3-1.26.18-py2.py3-none-any.whl (143 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.1.0
    Uninstalling urllib3-2.1.0:
      Successfully uninstalled urllib3-2.1.0
  Attempting uninstall: botocore
    Found existing installation: botocore 1.33.9
    Uninstalling botocore-1.33.9:
      Successfully uninstalled botocore-1.33.9
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0

In [3]:
# General
import pandas as pd
from pandas.tseries.offsets import MonthEnd
from datetime import datetime, timedelta
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import os
import numpy as np
import xlsxwriter
import datetime
import boto3
import s3fs
from itertools import combinations
import pickle

# Sklearn
from sklearn.model_selection import train_test_split, KFold
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer


# Models
from catboost import CatBoostClassifier, cv, Pool
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor


# Plots
import matplotlib.pyplot as plt
import seaborn as sns

# SHAP
import shap

# Random
import random

#Warnings
import warnings
warnings.filterwarnings("ignore")

# Utils

In [21]:
def inv_logit(x):
    return 1 / (1 + np.exp(-x))

def calculate_SHAP_and_probability_binary(model_promoter, model_detractor, df):
    # Extraer ID y fechas, manteniendo el índice
    id_df = df[['respondent_id', 'date_flight_local']]
    
    # Preparar el conjunto de datos para predicciones, excluyendo ID y fechas
    test_set = df.drop(['respondent_id', 'date_flight_local'], axis=1, errors='ignore')
    
    # Predicciones y probabilidades para promotores
    promoter_test_set = test_set.drop(['promoter_binary'], axis=1, errors='ignore')
    predictions_promoter = pd.DataFrame(model_promoter.predict(promoter_test_set), index=promoter_test_set.index, columns=["prediction_prom"])
    proba_promoter = pd.DataFrame(model_promoter.predict_proba(promoter_test_set)[:, 1], index=promoter_test_set.index, columns=["out_prob_prom"])
    
    # Predicciones y probabilidades para detractores
    detractor_test_set = test_set.drop(['detractor_binary'], axis=1, errors='ignore')
    predictions_detractor = pd.DataFrame(model_detractor.predict(detractor_test_set), index=detractor_test_set.index, columns=["prediction_det"])
    proba_detractor = pd.DataFrame(model_detractor.predict_proba(detractor_test_set)[:, 1], index=detractor_test_set.index, columns=["out_prob_det"])
    
    # Combinar resultados de predicción, manteniendo el índice original
    prediction = pd.concat([id_df, test_set, predictions_promoter, proba_promoter, predictions_detractor, proba_detractor], axis=1)
    
    # SHAP values y explicadores para el modelo promotor
    shap_Explainer_promoter = shap.TreeExplainer(model_promoter)
    shap_values_promoter = shap_Explainer_promoter.shap_values(promoter_test_set)
    feature_names = [i for i in promoter_test_set.columns]
    shap_values_prom = pd.DataFrame(shap_values_promoter, index=promoter_test_set.index, columns=[f"{i}_prom" for i in feature_names])
    shap_values_prom["base_value_prom"] = shap_Explainer_promoter.expected_value
    shap_values_prom["out_value_prom"] = shap_values_prom.sum(axis=1)
    
    # SHAP values y explicadores para el modelo detractor
    shap_Explainer_detractor = shap.TreeExplainer(model_detractor)
    shap_values_detractor = shap_Explainer_detractor.shap_values(detractor_test_set)
    shap_values_det = pd.DataFrame(shap_values_detractor, index=detractor_test_set.index, columns=[f"{i}_det" for i in feature_names])
    shap_values_det["base_value_det"] = shap_Explainer_detractor.expected_value
    shap_values_det["out_value_det"] = shap_values_det.sum(axis=1)
    
    # Combinar SHAP values con predicciones, manteniendo el índice original
    output_df = pd.concat([prediction, shap_values_prom, shap_values_det], axis=1)
    
    # Devolver el dataframe de salida
    return output_df


def from_shap_to_probability_binary(df, features_dummy, label_binary):
    output_df = df.copy()
    
    # Determinar el sufijo basado en el tipo de modelo (promoter o detractor)
    class_suffix = '_prom' if label_binary == 'promoter_binary' else '_det'
    
    # Identificar columnas de SHAP para la clase de interés, asumiendo que ya tienen el sufijo correcto
    shap_columns = [col for col in df.columns if col.endswith(class_suffix)]
    base_value_col = f'base_value{class_suffix}'
    
    # Convertir el valor base a probabilidades y actualizar el nombre de la columna
    output_df[f'base_prob{class_suffix}'] = inv_logit(output_df[base_value_col])
    
    # Convertir valores SHAP a probabilidades sin cambiar los nombres de las columnas
    for col in shap_columns:
        output_df[col] = inv_logit(output_df[col])
    
    # Asegurarse de incluir solo las columnas relevantes en el DataFrame final
    relevant_columns = ['respondent_id', 'date_flight_local'] + shap_columns + [f'base_prob{class_suffix}'] + features_dummy
    output_df = output_df[relevant_columns]
    return output_df

def adjust_shap_values_binary(shap_values, base_prob, out_prob):
    """Ajustar los valores SHAP para un modelo binario basado en la distancia."""
    # Calcular la distancia total deseada entre la probabilidad base y la de salida
    total_distance = out_prob - base_prob
    # Calcular la suma total de los valores SHAP
    total_shap = np.sum(shap_values)
    # Calcular el factor de ajuste si la suma total de SHAP no es cero
    adjustment_factor = total_distance / total_shap if total_shap != 0 else 0
    # Ajustar los valores SHAP
    return shap_values * adjustment_factor

def from_shap_to_probability_binary(df, features_dummy, label_binary):
    output_df = df.copy()
    
    # Determinar el sufijo basado en el tipo de modelo (promoter o detractor)
    class_suffix = '_prom' if label_binary == 'promoter_binary' else '_det'
    
    # Identificar columnas de SHAP para la clase de interés, asumiendo que ya tienen el sufijo correcto
    shap_columns = [f'{feature}{class_suffix}' for feature in features_dummy if f'{feature}{class_suffix}' in df.columns]
    base_value_col = f'base_value{class_suffix}'
    out_prob_col = f'out_prob{class_suffix}'

    # Calcular la probabilidad base usando softmax o inv_logit según sea apropiado
    output_df[f'base_prob{class_suffix}'] = inv_logit(output_df[base_value_col])

    for index, row in output_df.iterrows():
        # Extraer los valores SHAP para ajustar
        shap_values = row[shap_columns].values
        # Calcular los valores SHAP ajustados
        adjusted_shap_values = adjust_shap_values_binary(shap_values, row[f'base_prob{class_suffix}'], row[out_prob_col])
        # Actualizar el DataFrame con los valores SHAP ajustados
        output_df.loc[index, shap_columns] = adjusted_shap_values

    # Incluir solo las columnas relevantes en el DataFrame final
    relevant_columns = ['respondent_id', 'date_flight_local'] + shap_columns + [f'base_prob{class_suffix}', out_prob_col] + features_dummy
    output_df = output_df[relevant_columns]
    
    return output_df

def predict_and_explain(model_prom, model_det, df, features_dummy):
    """
    Realiza predicciones y genera explicaciones para modelos de promotores y detractores
    para todo el dataframe.

    Args:
    - model_prom: Modelo entrenado para predecir promotores.
    - model_det: Modelo entrenado para predecir detractores.
    - df: DataFrame con los datos.
    - features_dummy: Lista de características utilizadas para las predicciones.

    Returns:
    - Df final con .data, .values, .base_value, y predicciones.
    """
    # 1. Asumiendo que las funciones de cálculo de SHAP y probabilidad ya están implementadas y ajustadas para usar df
    df_contrib = calculate_SHAP_and_probability_binary(model_prom, model_det, df)

    # 3. Convertir valores SHAP a probabilidad
    df_probability_prom = from_shap_to_probability_binary(df_contrib, features_dummy, 'promoter_binary')
    df_probability_det = from_shap_to_probability_binary(df_contrib, features_dummy, 'detractor_binary')

    # 4. Concatenar DataFrames para ambos modelos
    df_probability_prom = df_probability_prom.reset_index(drop=True)
    df_probability_det = df_probability_det.reset_index(drop=True)
    unique_columns_det = [col for col in df_probability_det.columns if col not in df_probability_prom.columns]
    df_probability_binary = pd.concat([df_probability_prom, df_probability_det[unique_columns_det]], axis=1)

    # 5. Calcular columnas NPS con la diferencia entre _prom y _det
    for column in df_probability_binary.columns:
        if '_prom' in column:
            base_name = column.split('_prom')[0]
            det_column = f'{base_name}_det'
            if det_column in df_probability_binary.columns:
                nps_column = f'{base_name}_nps'
                df_probability_binary[nps_column] = df_probability_binary[column] - df_probability_binary[det_column]

    return df_probability_binary


In [12]:
def inv_logit(x):
    return 1 / (1 + np.exp(-x))

def calculate_SHAP_and_probability_binary(model_promoter, model_detractor, test_set):
    # Predicciones para el modelo de promotores
    promoter_test_set = test_set.drop(['promoter_binary'], axis=1, errors='ignore')
    predictions_promoter = pd.DataFrame(model_promoter.predict(promoter_test_set), columns=["prediction_prom"])
    proba_promoter = pd.DataFrame(model_promoter.predict_proba(promoter_test_set))[[1]].rename(columns={1: "out_prob_prom"})
    
    # Predicciones para el modelo de detractores
    detractor_test_set = test_set.drop(['detractor_binary'], axis=1, errors='ignore')
    predictions_detractor = pd.DataFrame(model_detractor.predict(detractor_test_set), columns=["prediction_det"])
    proba_detractor = pd.DataFrame(model_detractor.predict_proba(detractor_test_set))[[1]].rename(columns={1: "out_prob_det"})
    
    # Combinar resultados de predicción
    prediction = pd.concat([predictions_promoter, proba_promoter, predictions_detractor, proba_detractor, test_set.reset_index(drop=True)], axis=1)
    
    # SHAP values y explicadores para el modelo promotor
    shap_Explainer_promoter = shap.TreeExplainer(model_promoter)
    shap_values_promoter = shap_Explainer_promoter.shap_values(promoter_test_set)
    feature_names = [i for i in promoter_test_set.columns]
    shap_values_prom = pd.DataFrame(shap_values_promoter, columns=[f"{i}_prom" for i in feature_names])
    shap_values_prom["base_value_prom"] = shap_Explainer_promoter.expected_value
    shap_values_prom["out_value_prom"] = shap_values_prom.sum(axis=1)
    
    # SHAP values y explicadores para el modelo detractor
    shap_Explainer_detractor = shap.TreeExplainer(model_detractor)
    shap_values_detractor = shap_Explainer_detractor.shap_values(detractor_test_set)
    shap_values_det = pd.DataFrame(shap_values_detractor, columns=[f"{i}_det" for i in feature_names])
    shap_values_det["base_value_det"] = shap_Explainer_detractor.expected_value
    shap_values_det["out_value_det"] = shap_values_det.sum(axis=1)
    
    # Combinar SHAP values con predicciones
    output_df = pd.concat([prediction, shap_values_prom, shap_values_det], axis=1)
    
    # Devolver el dataframe de salida y los explicadores SHAP
    return output_df, shap_Explainer_promoter, shap_Explainer_detractor

def from_shap_to_probability_binary(df, features_dummy, label_binary):
    output_df = df.copy()
    
    # Determinar el sufijo basado en el tipo de modelo (promoter o detractor)
    class_suffix = '_prom' if label_binary == 'promoter_binary' else '_det'
    
    # Identificar columnas de SHAP para la clase de interés, asumiendo que ya tienen el sufijo correcto
    shap_columns = [col for col in df.columns if col.endswith(class_suffix)]
    base_value_col = f'base_value{class_suffix}'
    
    # Convertir el valor base a probabilidades y actualizar el nombre de la columna
    output_df[f'base_prob{class_suffix}'] = inv_logit(output_df[base_value_col])
    
    # Convertir valores SHAP a probabilidades sin cambiar los nombres de las columnas
    for col in shap_columns:
        output_df[col] = inv_logit(output_df[col])
    
    # Asegurarse de incluir solo las columnas relevantes en el DataFrame final
    relevant_columns = shap_columns + [f'base_prob{class_suffix}'] + features_dummy
    output_df = output_df[relevant_columns]
    
    return output_df

def adjust_shap_values_binary(shap_values, base_prob, out_prob):
    """Ajustar los valores SHAP para un modelo binario basado en la distancia."""
    # Calcular la distancia total deseada entre la probabilidad base y la de salida
    total_distance = out_prob - base_prob
    # Calcular la suma total de los valores SHAP
    total_shap = np.sum(shap_values)
    # Calcular el factor de ajuste si la suma total de SHAP no es cero
    adjustment_factor = total_distance / total_shap if total_shap != 0 else 0
    # Ajustar los valores SHAP
    return shap_values * adjustment_factor

def from_shap_to_probability_binary(df, features_dummy, label_binary):
    output_df = df.copy()
    
    # Determinar el sufijo basado en el tipo de modelo (promoter o detractor)
    class_suffix = '_prom' if label_binary == 'promoter_binary' else '_det'
    
    # Identificar columnas de SHAP para la clase de interés, asumiendo que ya tienen el sufijo correcto
    shap_columns = [f'{feature}{class_suffix}' for feature in features_dummy if f'{feature}{class_suffix}' in df.columns]
    base_value_col = f'base_value{class_suffix}'
    out_prob_col = f'out_prob{class_suffix}'

    # Calcular la probabilidad base usando softmax o inv_logit según sea apropiado
    output_df[f'base_prob{class_suffix}'] = inv_logit(output_df[base_value_col])

    for index, row in output_df.iterrows():
        # Extraer los valores SHAP para ajustar
        shap_values = row[shap_columns].values
        # Calcular los valores SHAP ajustados
        adjusted_shap_values = adjust_shap_values_binary(shap_values, row[f'base_prob{class_suffix}'], row[out_prob_col])
        # Actualizar el DataFrame con los valores SHAP ajustados
        output_df.loc[index, shap_columns] = adjusted_shap_values

    # Incluir solo las columnas relevantes en el DataFrame final
    relevant_columns = shap_columns + [f'base_prob{class_suffix}', out_prob_col] + features_dummy
    output_df = output_df[relevant_columns]
    
    return output_df

def predict_and_explain(model_prom, model_det, df, features_dummy, start_date, end_date):
    """
    Realiza predicciones y genera explicaciones para modelos de promotores y detractores
    dentro de un rango de fechas específico.

    Args:
    - model_prom: Modelo entrenado para predecir promotores.
    - model_det: Modelo entrenado para predecir detractores.
    - df: DataFrame con los datos.
    - features_dummy: Lista de características utilizadas para las predicciones.
    - start_date: Fecha de inicio para los datos de prueba (formato 'YYYY-MM-DD').
    - end_date: Fecha de fin para los datos de prueba (formato 'YYYY-MM-DD').

    Returns:
    - Nada, pero guarda los objetos de explicación SHAP con nombres que reflejan el mes y año.
    """
    # 1. Filtrar el DataFrame por el rango de fechas
    df_filtered = df[(df['date_flight_local'] >= start_date) & (df['date_flight_local'] < end_date)]

    # 2. Asumiendo que las funciones de cálculo de SHAP y probabilidad ya están implementadas y ajustadas para usar df_filtered
    df_contrib, explainer_prom, explainer_det = calculate_SHAP_and_probability_binary(model_prom, model_det, df_filtered[features_dummy])

    # 3. Convertir valores SHAP a probabilidad
    df_probability_prom = from_shap_to_probability_binary(df_contrib, features_dummy, 'promoter_binary')
    df_probability_det = from_shap_to_probability_binary(df_contrib, features_dummy, 'detractor_binary')

    # 4. Concatenar DataFrames para ambos modelos
    df_probability_prom = df_probability_prom.reset_index(drop=True)
    df_probability_det = df_probability_det.reset_index(drop=True)
    unique_columns_det = [col for col in df_probability_det.columns if col not in df_probability_prom.columns]
    df_probability_binary = pd.concat([df_probability_prom, df_probability_det[unique_columns_det]], axis=1)

    # 5. Calcular columnas NPS con la diferencia entre _prom y _det
    for column in df_probability_binary.columns:
        if '_prom' in column:
            base_name = column.split('_prom')[0]
            det_column = f'{base_name}_det'
            if det_column in df_probability_binary.columns:
                nps_column = f'{base_name}_nps'
                df_probability_binary[nps_column] = df_probability_binary[column] - df_probability_binary[det_column]

    # 6. Agregar variables y valores SHAP para crear una explicación general
    # num_vars = ['delay_departure','ticket_price']
    num_vars = ['ticket_price', 'load_factor']
    bin_vars = ['otp15_takeoff']
    # bin_vars = ['otp15_takeoff'] + [col for col in df_nps_tkt.columns if 'country_agg' in col]
    # cat_vars=['segment']
    # bin_vars=[]
    cat_vars=[]
    touchpoints = [feat for feat in features_dummy if feat not in bin_vars + num_vars]
    values_nps_sum = [df_probability_binary[f'{feat}_nps'].mean()*100 for feat in features_dummy]
    num_var_scores = [df_probability_binary[num_var].mean() for num_var in num_vars]
    bin_vars_scores = []
    for var in bin_vars:
        if var == 'otp15_takeoff':
            # Para 'otp15_takeoff', calcula el porcentaje de 0s.
            score = (df_probability_binary[df_probability_binary[var] == 0][var].count() / 
                     df_probability_binary[var].count()) * 100
        else:
            # Para cualquier otra variable, calcula el porcentaje de 1s.
            score = (df_probability_binary[df_probability_binary[var] == 1][var].count() / 
                     df_probability_binary[var].count()) * 100
        bin_vars_scores.append(score)
    # Continuación después de calcular bin_vars_scores
    cat_vars_scores = []
    for cat_var in cat_vars:
        cat_vars_scores.append(0)

    satisfaction_scores = [df_probability_binary[df_probability_binary[tp] >= 8][tp].count() / df_probability_binary[tp].count() * 100 for tp in touchpoints]
    shap_data = np.array(num_var_scores + bin_vars_scores + satisfaction_scores)
    base_value_nps_sum = df_probability_binary['base_prob_prom'].mean() * 100 - df_probability_binary['base_prob_det'].mean() * 100
    shap_values = np.array(values_nps_sum)  # Convertimos la lista en un array 2D
    features_names = np.array(features_dummy)
    explainer = shap.Explanation(values=shap_values, 
                                 base_values=base_value_nps_sum, 
                                 data=shap_data, 
                                 feature_names=features_names)
        
    return explainer, df_probability_binary

# Predict

In [22]:
features = ['ticket_price', 'load_factor', 'otp15_takeoff', 'bkg_200_journey_preparation', 'pfl_100_checkin', 
                  'pfl_200_security', 'pfl_300_lounge', 'pfl_500_boarding', 'ifl_300_cabin', 
                  'ifl_200_flight_crew_annoucements', 'ifl_600_wifi', 'ifl_500_ife', 'ifl_400_food_drink', 
                  'ifl_100_cabin_crew', 'arr_100_arrivals', 'con_100_connections', 
                  'loy_200_loyalty_programme', 'img_310_ease_contact_phone']

In [23]:
model_names=['PROM','DET']
clf_model={}
for name in model_names:
    path_model=f'pipeline_output/CatBoostClassifier_cv_{name}.pkl'
    # Cargar el modelo desde el archivo .pkl
    with open(path_model, 'rb') as file:
        clf_model[name] = pickle.load(file)

In [24]:
# Load the data to predict
df_predict = pd.read_csv(f"pipeline_output/data_for_historic_prediction.csv")
    
# Asegurarse de que 'date_flight_local' esté en formato datetime
df_predict['date_flight_local'] = pd.to_datetime(df_predict['date_flight_local'])
df_predict = df_predict[df_predict['date_flight_local'].dt.year == 2023]
    
df_predict = df_predict[df_predict['date_flight_local'].dt.month == 1]

def filter_data_by_quarter(df, quarter):
    # Definir los rangos de fechas para cada trimestre
    quarters = {
        "q1": (1, 3),
        "q2": (4, 6),
        "q3": (7, 9),
        "q4": (10, 12)
    }

    # Obtener el rango de meses para el trimestre especificado
    start_month, end_month = quarters[quarter]

    # Filtrar el DataFrame por el rango de fechas del trimestre
    df_filtered = df[df['date_flight_local'].dt.month.between(start_month, end_month)]

    return df_filtered

quarters = ['q1']

for quarter in quarters:
    df_predict = filter_data_by_quarter(df_predict, quarter)
    # Perform prediction and add the probabilities to the dataframe
    test_set = df_predict.drop(['respondent_id'], axis=1, errors='ignore')
    df_probabilities = predict_and_explain(clf_model[model_names[0]], clf_model[model_names[1]], df_predict, features)
    # explainer, df_probabilities = predict_and_explain(clf_model[model_names[0]], clf_model[model_names[1]], test_set, features, '2023-01-01','2023-01-31')


In [25]:
df_probabilities['out_prob_nps'].mean()

0.3086387843699623

In [15]:
df_probabilities['out_prob_nps'].mean()

0.05299504376253408

In [ ]:

# # Rename columns, add insert date and select columns to save
# df_probabilities['insert_date_ci'] = STR_EXECUTION_DATE
# df_probabilities['model_version']=f'{model_year}-{model_month}-{model_day}'
# df_probabilities = df_probabilities[config['PREDICT']['COLUMNS_SAVE']]

# Save the prediction results to S3
df_probabilities.to_csv(save_path, index=False)